In [1]:
import numpy as np
import torch
import scipy
import random
import matplotlib.pyplot as plt
import time

In [2]:
# set wavenumber and wavelength
k = 2.0 * np.pi
lamb =  2.0 * np.pi / k

# domain [la,lb]
la = -12.0 * lamb
lb = 12.0 * lamb

# define the mesh for calculation
N_obs = 240
h_obs = (lb - la) / N_obs
mesh_obs = np.linspace(la, lb, (N_obs + 1))

mesh_mid_obs = np.zeros(N_obs + 1)

mesh_mid_obs[0] = la

for od in range(N_obs):
    mesh_mid_obs[od + 1] = (mesh_obs[od] + mesh_obs[od+1])/2.0

# transform numpy mesh to torch mesh
x_mid_obs = torch.tensor(mesh_mid_obs, dtype = torch.float32, requires_grad = True)
x_mesh_obs = torch.tensor(mesh_obs, dtype = torch.float32, requires_grad = True)

In [3]:
# generate random rough surfaces

def generate_h():
    '''this function generates a scaled and tappered Gaussian type rough surface'''
    
    # define the a.c.f. rho(xi)
    def rho(xi):
        lamb_surface = lamb 
        insf = -(xi ** 2) / (lamb_surface ** 2)
        return np.exp(insf)

    # the following code generates a random rough surface w.r.t. the a.c.f rho
    def fft(xi,mu):
        return 2.0 / np.pi * rho(xi) * np.cos(xi * mu)

    Nh = 50
    muend = 12
    deltamu = muend / Nh
    mu_vec = np.linspace(deltamu, muend, Nh)
    integralB = np.zeros(Nh)
    integralA = np.zeros(Nh)

    for it in range(Nh):
        def fg(xi):
            return fft(xi,mu_vec[it])
        integralB[it] = scipy.integrate.quad(fg,-scipy.inf,scipy.inf)[0]
        integralA[it] = np.sqrt(integralB[it])
        
    fai = np.zeros(Nh)
    for ii in range(Nh):
        fai[ii] = 2.0 * np.pi * random.random()

    # original surface height
    def hei_ori(x):
        sumn = 0.0
        for js in range(Nh):
            sumn = sumn + integralA[js] * np.sin(mu_vec[js] * x + fai[js])
        return np.sqrt(deltamu) * sumn 
    
    # derivative of original surface
    def heid_ori(x):
        sumn2 = 0.0
        for js in range(Nh):
            sumn2 = sumn2 + integralA[js] * mu_vec[js] * np.cos(mu_vec[js] * x + fai[js])
        return np.sqrt(deltamu) * sumn2 
    
    # second order derivative of original surface
    def heidd_ori(x):
        sumn3 = 0.0
        for js in range(Nh):
            sumn3 = sumn3 - integralA[js] * mu_vec[js] * mu_vec[js] * np.sin(mu_vec[js] * x + fai[js])
        return np.sqrt(deltamu) * sumn3 

    # scale surface, set surface height in [hmin, hmax]
    function_values = hei_ori(mesh_obs)
    fmax = np.max(function_values)
    fmin = np.min(function_values)
    hmax = 0.2 * lamb
    hmin = -0.2 * lamb
    
    # scaled rough surface
    def hei_surface(x):
        return (hmax - hmin) * (hei_ori(x) - fmin) / (fmax - fmin) + hmin
    
    # tappering the surface to zero close boundaries
    a_tape = 0.8 * la
    b_tape = 0.8 * lb
    factor_tape = 5
    
    def function_tapper(x):
        tapper_left = 0.5 * (np.tanh(factor_tape * (x - a_tape)) + 1.0)
        tapper_right = 0.5 * (np.tanh(factor_tape * (b_tape - x)) + 1.0)
        return tapper_left * tapper_right
    
    # tappered surface
    def surface_tapper(x):
        return hei_surface(x) * function_tapper(x)
    
    # assemble surface height
    hei_vec = surface_tapper(mesh_mid_obs)
    
    # assemble surface derivatives
    heid_vec = np.zeros(N_obs + 1)
    for j in range(1, N_obs):
        heid_vec[j] = (hei_vec[j+1] - hei_vec[j]) / h_obs
        
    heidd_vec = np.zeros(N_obs + 1)
    for j in range(1, N_obs):
        heidd_vec[j] = (heid_vec[j+1] - heid_vec[j]) / h_obs
    
    hei_surface_true_torch = torch.tensor(hei_vec, dtype = torch.float32)
    heid_surface_true_torch = torch.tensor(heid_vec, dtype = torch.float32)
    heidd_surface_true_torch = torch.tensor(heidd_vec, dtype = torch.float32)
    
    return hei_surface_true_torch, heid_surface_true_torch, heidd_surface_true_torch

In [4]:
# the incident wave, which is a plane wave
def inc_field(x, z, alpha):
    g = torch.cos(alpha) * x + torch.sin(alpha) * z
    return torch.exp(1.0 * 1j * k * g)
    
# Hankel function of first order of first kind
# which is needed in the Green's function for Neumann case
def hankel(sth):
    return torch.special.bessel_j1(sth) + 1j * torch.special.bessel_y1(sth)
    
# method of moments
# this function produces the scattered field
def calculate(x_mid, hei_vec, heid_vec, heidd_vec, theta1, zz):
    
    ''' This function implements the method of moments for the Neumann case.
     x_mid is the mesh points.
     hei_vec is the torch tensor for the surface height.
     heid_vec is the torch tensor of the derivative of surface height.
     it reaturns the torch tensor of scattered field.'''
    
    
    h = x_mid[100] - x_mid[99]
    N = x_mid.numel() - 1
    
    # aseemble surface incident field
    inc_vec =  torch.zeros(N, dtype=torch.cfloat)
    for i in range(N):
        inc_vec[i] = inc_field(x_mid[i+1], hei_vec[i+1], theta1)
    
    # aseemble matrix A
    A =  torch.zeros((N, N), dtype=torch.cfloat)
    
    # off-diagonal terms
    def getA_offdiag1(nn):
        ll = torch.arange(1, nn)
        ptl = torch.sqrt((x_mid[nn] - x_mid[ll]) ** 2 + (hei_vec[nn] - hei_vec[ll])**2)
        inte1 = hankel(k * ptl) / ptl
        inte2 = -1.0 * heid_vec[ll] * (x_mid[nn] - x_mid[ll]) + (hei_vec[nn] - hei_vec[ll])
        sd = torch.sqrt(1.0 + heid_vec[ll] ** 2)
        return 1.0 * k * 1j / 4.0 * h * inte1 * inte2 * sd 
    def getA_offdiag2(nn):
        ll = torch.arange(nn + 1, N + 1)
        ptl = torch.sqrt((x_mid[nn] - x_mid[ll]) ** 2 + (hei_vec[nn] - hei_vec[ll])**2)
        inte1 = hankel(k * ptl) / ptl
        inte2 = -1.0 * heid_vec[ll] * (x_mid[nn] - x_mid[ll]) + (hei_vec[nn] - hei_vec[ll])
        sd = torch.sqrt(1.0 + heid_vec[ll] ** 2)
        return 1.0 * k * 1j / 4.0 * h * inte1 * inte2 * sd 
    
    # diagonal terms
    def getA_diag(nn):
        return 1.0 * h * k * heidd_vec[nn] / ((1.0 + heid_vec[nn] ** 2) * 4 * torch.pi)

    # aseemble matrix A
    for nu in range(N):
        A[nu, 0:nu] = -1.0 * getA_offdiag1(nu + 1)
        A[nu, nu+1:N] = -1.0 * getA_offdiag2(nu + 1)
        A[nu, nu] = -1.0 * getA_diag(nu + 1) + torch.tensor([0.5])
    
    # solve for surface current
    phid = torch.linalg.solve(A, inc_vec)
    
    # assemble matrix B
    B =  torch.zeros((N, N), dtype=torch.cfloat)

    # components of matrix B
    def getB(nn):
        rr = torch.arange(1, N + 1)
        ptl = torch.sqrt((x_mid[nn] - x_mid[rr]) ** 2 + (zz - hei_vec[rr])**2)
        inte1 = hankel(k * ptl) / ptl
        inte2 = -1.0 * heid_vec[rr] * (x_mid[nn] - x_mid[rr]) + (zz - hei_vec[rr])
        sd = torch.sqrt(1.0 + heid_vec[rr] ** 2)
        return 1.0 * k * h * 1j / 4.0 * inte1 * inte2 * sd 
    
    for nd in range(N):
        B[nd, 0:N] = getB(nd + 1)

    # calculate scattered field
    phis = torch.matmul(B, phid)
    
    # return the scattered field
    return phis

In [5]:
# save the surface height, measurement height, and scattered field in a list
data_list = []

# set the incident angle
alpha_use = torch.tensor([[-1.0 * np.pi / 4.0]])

# number of surface data generated
# note that for each surface, we set 5 values of measurement height, 
# thus total number of data is Ndata * 5 
Ndata = 200

time1 = time.time()
for i in range(Ndata):
    
    # generate a random rough surface
    hei_surface_true, heid_surface_true, heidd_surface_true = generate_h()
    
    # quasi-random sampling of measurement height
    zz1 = random.uniform(0.45,0.55)
    zz2 = random.uniform(0.55,0.65)
    zz3 = random.uniform(0.65,0.75)
    zz4 = random.uniform(0.75,0.85)
    zz5 = random.uniform(0.85,0.95)
    zz_all = [zz1,zz2,zz3,zz4,zz5]
    
    for zs in zz_all:
            
        zz_use = torch.tensor([zs * lamb])
    
        # MOM
        phis_line = calculate(x_mid_obs, hei_surface_true, heid_surface_true, heidd_surface_true, alpha_use, zz_use)
        
        hei_surface_use = hei_surface_true[1:]
    
        data_here = [hei_surface_use, phis_line, zz_use]
    
        data_list.append(data_here)
        
time2 = time.time()
print(time2 - time1)

199.63188362121582


In [6]:
# save the data list in .pt file supported by PyTorch
torch.save(data_list, 'data20_neu.pt')